# 1. Importation des librairies et chargement du dataset

In [38]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Masking, Embedding, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn import metrics
from sklearn import linear_model
from keras.models import Sequential
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import requests
import json

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [16]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


#Selection de la data

In [17]:
data = pd.read_csv('gdrive/MyDrive/Projet_NLP/dataset.csv',encoding='ISO-8859-1',header=None,names=['target','ids','date','flag','user','text'])

dataset_neg=data['text'][:10000]
dataset_pos=data['text'][800000:810000]

sentiment_neg=data['target'][:10000]
sentiment_pos=data['target'][800000:810000]


#Mise en DataFrame + suppression de la ponctuation superflue

In [18]:
def to_tab(data):
  sentiment_tab=[]
  for string_line in data:
    sentiment_tab.append(string_line)

  return sentiment_tab

dataset_sentiment_pos = to_tab(sentiment_pos)
dataset_sentiment_neg = to_tab(sentiment_neg)

dataset_sentiment= dataset_sentiment_pos + dataset_sentiment_neg




def dataset_nopunct(data):
  punct = string.punctuation
  tab_punct=[]
  for i in range(len(punct)):
    tab_punct.append(punct[i])


  no_punct_dataset=[]
  element=""
  for elt in data:
    for letter in elt:
      if letter not in tab_punct and not letter.isdigit():
        element += letter

    no_punct_dataset.append(element)
    element =""
  
  return no_punct_dataset

datasetPosNoPunct=dataset_nopunct(dataset_pos)
datasetNegNoPunct=dataset_nopunct(dataset_neg)


datasetNoPunct=dataset_nopunct(dataset_pos) +  dataset_nopunct(dataset_neg)
tab=pd.DataFrame(datasetNoPunct, columns = ["text"])
tab["sentiment"] = dataset_sentiment
tab

,text,sentiment
0,I LOVE HealthUandPets u guys r the best,4
1,im meeting up with one of my besties tonight C...,4
2,DaRealSunisaKim Thanks for the Twitter add Sun...,4
3,Being sick can be really cheap when it hurts t...,4
4,LovesBrooklyn he has that effect on everyone,4
...,...,...
19995,Aww thats sad,0
19996,stupid dvds stuffing up the good bits in jaws,0
19997,DandySephy No Only close friends and family Im...,0
19998,CRAP After looking when I last tweeted WHY AM ...,0


#Mise en minuscule des tweets

In [19]:
def to_lower(data):
  for i in range(len(data["text"])):
    data["text"][i]=data["text"][i].lower()
  
  return data

dataset_lower=to_lower(tab)
dataset_lower

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,sentiment
0,i love healthuandpets u guys r the best,4
1,im meeting up with one of my besties tonight c...,4
2,darealsunisakim thanks for the twitter add sun...,4
3,being sick can be really cheap when it hurts t...,4
4,lovesbrooklyn he has that effect on everyone,4
...,...,...
19995,aww thats sad,0
19996,stupid dvds stuffing up the good bits in jaws,0
19997,dandysephy no only close friends and family im...,0
19998,crap after looking when i last tweeted why am ...,0


#Tokenization des tweets avec keras

In [20]:
#The Tokenizer class of Keras is used for vectorizing a text corpus.
#For this either, each text input is converted into integer sequence or a vector that has a coefficient for each token in the form of binary values.
tokenizer = Tokenizer(num_words=20000, split=" ")

#The fit_on_texts method is a part of Keras tokenizer class which is used to update the internal vocabulary for the texts list.
#We need to call be before using other methods of texts_to_sequences or texts_to_matrix.
tokenizer.fit_on_texts(dataset_lower["text"].values)

#texts_to_sequences method helps in converting tokens of text corpus into a sequence of integers.
X= tokenizer.texts_to_sequences(dataset_lower["text"].values)

#pad_sequence takes a LIST of sequences as an input (list of list) and will return a list of padded sequences (all the same length).
X = pad_sequences(X)
print(X[:33])
print(X.shape)

[[   0    0    0 ...  388    3  179]
 [   0    0    0 ...  142  294  324]
 [   0    0    0 ...  131    4 2092]
 ...
 [   0    0    0 ...   25    3  422]
 [   0    0    0 ...  712   18  109]
 [   0    0    0 ... 9500 9501 9502]]
(20000, 32)


In [21]:
# 4 --> [0 1] pour les positifs et 0 --> [1 0] pour les négatifs
Y=pd.get_dummies(tab["sentiment"]).values
print(Y)
print(Y.shape)


[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]
(20000, 2)


##Modèle de RNN

In [22]:

model = Sequential()

#Transforme les entiers positifs (index) en vecteurs de taille fixe.
#La première étape de l'utilisation d'une couche d'incorporation consiste à encoder cette phrase par des indices.
model.add(Embedding(20000, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))

#Les réseaux de longue mémoire à court terme (LSTM) sont une extension pour les réseaux neuronaux récurrents, qui étend leur mémoire.
#Par conséquent, il est bien adapté pour apprendre des expériences importantes qui ont des retards très longs.
model.add(LSTM(256,return_sequences=False,dropout=0.3,recurrent_dropout=0.2))

#Sigmoïde: produit une courbe en forme de S.
#Bien que de nature non linéaire, il ne tient toutefois pas compte des légères variations des entrées, ce qui entraîne des résultats similaires.
model.add(Dense(2, activation="sigmoid"))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

#On split la data, 70% pour l'entrainement et le reste pour la validation
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.7,random_state=0)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 256)           5120000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 5,645,826
Trainable params: 5,645,826
Non-trainable params: 0
_________________________________________________________________


##Entrainement du modèle

In [23]:
model.fit(x_train,  y_train, 
                   epochs=8, batch_size=100,verbose =2)

Epoch 1/8
140/140 - 87s - loss: 0.5860 - accuracy: 0.6859
Epoch 2/8
140/140 - 85s - loss: 0.4106 - accuracy: 0.8212
Epoch 3/8
140/140 - 85s - loss: 0.2973 - accuracy: 0.8811
Epoch 4/8
140/140 - 85s - loss: 0.2076 - accuracy: 0.9202
Epoch 5/8
140/140 - 85s - loss: 0.1412 - accuracy: 0.9452
Epoch 6/8
140/140 - 86s - loss: 0.1059 - accuracy: 0.9593
Epoch 7/8
140/140 - 86s - loss: 0.0816 - accuracy: 0.9686
Epoch 8/8
140/140 - 86s - loss: 0.0630 - accuracy: 0.9774


#Chargement du modèle déjà entrainé

In [ ]:
#model_1 = keras.models.load_model('/content/gdrive/MyDrive/Projet_NLP/Reseau_RNN_model_1/Model_1')
#model_1.summary()

##Evaluation du modèle

In [24]:
model.evaluate(x_test,y_test)


188/188 [==============================] - 8s 43ms/step - loss: 1.1008 - accuracy: 0.7232


[1.100848913192749, 0.7231666445732117]

##Sauvegarde du modèle

In [27]:
#model.save('/content/gdrive/MyDrive/Projet_NLP/Reseau_RNN_models/Model_1')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Projet_NLP/Reseau_RNN_models/Model_1/assets


##Récupération de reviews depuis l'API Yelp

In [28]:
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}


url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


data=json.loads(req.text)
data

the status code is 200


{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=KzZVmcfbYqCMY9omoha5NA&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KzZVmcfbYqCMY9omoha5NA',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

#Traitement de nos futurs tweets/reviews qui nous serviront d'inputs

In [34]:
def dataset_nopunct(data):
    punct = string.punctuation
    tab_punct=[]
    for i in range(len(punct)):
      tab_punct.append(punct[i])


    no_punct_dataset=[]
    element=""
  
    for letter in data:
      if letter not in tab_punct and not letter.isdigit():
        element += letter

    no_punct_dataset.append(element)
    element =""
    
    return no_punct_dataset
    
def traitement(my_string):
    
  dataset_string=dataset_nopunct(my_string)

  tab_string=pd.DataFrame(dataset_string, columns = ["text"])

  test_lower=to_lower(tab_string)

  tokenizer = Tokenizer(num_words=20000, split=" ")
  tokenizer.fit_on_texts(test_lower["text"].values)

  X_string= tokenizer.texts_to_sequences(test_lower["text"].values)
  X_string = pad_sequences(X_string)

  return X_string



#Prediction

In [37]:
str1 =data["reviews"][0]["text"]
str2="I ordered a burger and milk shake thru Uber eats, only took a few bites of burger fries and a few sips of milk shake and got extremely ill after taking a few bites of the burger and a few sips of milkshake. My body could tell something was off and started feeling nauseous, had a horrible pounding headache and was violently vomiting.  I don't know if this was food poisoning or the food was tampered with , I   Will not be ordering from here again. Don't eat fast food often , but have never had a issue with shack shack before the other times I had it."
str3="Buying beware...these are addicting.Still the best doughnuts in my opinion in the city. Ginormous and fluffy like a pillow but somehow still a substantial bite.All doughnuts are scrumptious but greatly enjoyed the Dulce De De Leche in the batch we tried the best.Big win, you can order on Seamless for delivery also on the website they say Caviar, Doordash, Uber Eats and Grub Hub and Sweetist are among other sites for delivery as well."


test1 = traitement(str1)
test2 = traitement(str2)
test3 = traitement(str3)


preds1 = model.predict(test1)
preds2 = model.predict(test2)
preds3 = model.predict(test3)
print(preds1)
print(preds2)
print(preds3)


prediction_list=[preds1,preds2,preds3]
sentiment_target=["positif","negatif","positif"]

for i in range(len(prediction_list)):
  if prediction_list[i][0][0] < (1/3):
    print('predicted sentiment : negatif')
    print(f"target sentiment : {sentiment_target[i]}")
  elif  prediction_list[i][0][0] > (2/3): 
    print('predicted sentiment : positif')
    print(f"target sentiment : {sentiment_target[i]}")
  else:
    print('predicted sentiment : neutre')
    print(f"target sentiment : {sentiment_target[i]}")


[[0.9841034  0.01516014]]
[[0.10678524 0.88083184]]
[[0.88095653 0.12048572]]
predicted sentiment : positif
target sentiment : positif
predicted sentiment : negatif
target sentiment : negatif
predicted sentiment : positif
target sentiment : positif
